In [1]:
import os
import re
import pandas as pd
import numpy as np
import pickle as pickle

#### 사용법
1. prediction 폴더에 submission 파일들을 넣는다
2. submission 파일명 규칙은 모델명_submission.csv로 한다.

In [2]:
prediction_dir_file_list = os.listdir("./prediction")
prediction_dir_file_list

['copy_sample_submission.csv', '.ipynb_checkpoints', 'sample_submission.csv']

In [3]:
submission_list = []
for file in prediction_dir_file_list:
    if bool(re.findall("submission.csv",file)):
        submission_list.append(file)
submission_list

['copy_sample_submission.csv', 'sample_submission.csv']

In [33]:
n = len(submission_list)

probs = np.array([eval(x) for x in pd.read_csv(os.path.join('./prediction',submission_list[0]))['probs']])

for file in submission_list[1:]:
    submission = pd.read_csv(os.path.join('./prediction',file))
    probs += np.array([eval(x) for x in submission['probs']]) * (1/n)

probs

array([[0.15, 0.15, 0.15, ..., 0.15, 0.15, 0.15],
       [0.15, 0.15, 0.15, ..., 0.15, 0.15, 0.15],
       [0.15, 0.15, 0.15, ..., 0.15, 0.15, 0.15],
       ...,
       [0.15, 0.15, 0.15, ..., 0.15, 0.15, 0.15],
       [0.15, 0.15, 0.15, ..., 0.15, 0.15, 0.15],
       [0.15, 0.15, 0.15, ..., 0.15, 0.15, 0.15]])

In [34]:
result = np.argmax(probs, axis=-1)
result

array([0, 0, 0, ..., 0, 0, 0])

In [35]:
def num_to_label(label):
  """
    숫자로 되어 있던 class를 원본 문자열 라벨로 변환 합니다.
  """
  origin_label = []
  with open('dict_num_to_label.pkl', 'rb') as f:
    dict_num_to_label = pickle.load(f)
  for v in label:
    origin_label.append(dict_num_to_label[v])
  
  return origin_label

In [36]:
labels = num_to_label(result)

In [37]:
probs = [  '['+','.join([format(x,'.2f') for x in softmax])+']' for softmax in probs]
probs[0]

'[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15]'

In [38]:
output = pd.DataFrame({'id':submission['id'],'pred_label':labels,'probs':probs,})

In [39]:
output

,id,pred_label,probs
0,0,no_relation,"[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,..."
1,1,no_relation,"[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,..."
2,2,no_relation,"[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,..."
3,3,no_relation,"[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,..."
4,4,no_relation,"[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,..."
...,...,...,...
7760,7760,no_relation,"[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,..."
7761,7761,no_relation,"[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,..."
7762,7762,no_relation,"[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,..."
7763,7763,no_relation,"[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,..."


In [40]:
output.to_csv('./prediction/final_submission.csv', index=False)